In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format('parquet').option('header', 'true').option('path', 'abfss://bronze@adls5485.dfs.core.windows.net/products').load()

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView("products")


In [0]:
%sql

select * from products;

In [0]:
%sql

create or replace function sales_prj.bronze.discount_func(p_price double)
returns double
language sql
return p_price*0.90

In [0]:
df = df.withColumn('dicounted_price',expr('sales_prj.bronze.discount_func(price)'))
df.display()

In [0]:
%sql
create or replace function sales_prj.bronze.upper_func(p_brand string)
returns string
language sql
return upper(p_brand)


In [0]:
df = df.withColumn('brand',expr('sales_prj.bronze.upper_func(brand)'))
df.display()

In [0]:
df.write.format('delta').mode('overwrite').option('header','true').option('path','abfss://silver@adls5485.dfs.core.windows.net/products').save()

In [0]:
%sql

create table if not exists sales_prj.gold.products
using delta
location 'abfss://silver@adls5485.dfs.core.windows.net/products'